In [1]:
import sys
import spotipy
import spotipy.util as util

In [189]:
import os
import json
import urllib.request

## Auth

Setup Authentication. I had to do this a few times during the cource of working with the Spotify API

In [10]:
scope = 'user-library-read'


In [156]:
CLIENT_ID = ''
CLIENT_SECRET = ''

util.prompt_for_user_token('vlandham4', scope, client_id = CLIENT_ID, client_secret = CLIENT_SECRET, redirect_uri = 'http://localhost/')

'BQDkiMoeVtt3Abq6-vUPZbS_aMbmoo3iv8Zfl5A6ghMZFPVpGuA9IVLnM_XGw1xs5kTvNFVWUe9sYROTyjSAVFYudQDwLmuEU39CVEzrkjuskV3g_eJRZWTcJtNjryHt8VGOBeum_p4-ELkDDDjuBvg'

In [168]:

# copy auth from above. Probably should be stored in a variable.
sp = spotipy.Spotify(auth='BQDkiMoeVtt3Abq6-vUPZbS_aMbmoo3iv8Zfl5A6ghMZFPVpGuA9IVLnM_XGw1xs5kTvNFVWUe9sYROTyjSAVFYudQDwLmuEU39CVEzrkjuskV3g_eJRZWTcJtNjryHt8VGOBeum_p4-ELkDDDjuBvg')

Check if authentication worked

In [16]:
results = sp.current_user_saved_tracks()

In [18]:
for item in results['items']:
    track = item['track']
    print(track['name'] + ' - ' + track['artists'][0]['name'])

Jet - Paul McCartney
All Day All Night - River Whyless
Constellation - Waterstrider
From a Soon-to-Be Ghost Town - Fruit Bats
Genevieve - Lucius
Water Fountain - tUnE-yArDs
Get Your Shit Together - Pillow Fight
The Cause - Laura Gibson
Dancing at the Blue Lagoon - Cayucas
Get Low - Smooth Hound Smith
Lazer Eyes - Teenage Love
1986 (feat. MC Lyte) - Compass: Mexican Institute Of Sound + Toy Selectah
Oh Sweet Nuthin' - Hollis Brown
Edge of Town - Middle Kids
Half Crazy - The Barr Brothers
Melted Rainbow - Mr. Gnome
Clay Pigeons - Michael Cera
Breathe - ANIMA!
Fruit Trees - Matthew E. White
Out Of Pocket - Mayer Hawthorne


## Categories

First, lets grab all the categories we can.

Categories are the closest thing to Genre I could find

In [42]:
offset = 0
limit = 50
allCats = []
for i in range(2):
    cats = sp.categories(limit = 50, offset = offset)
    allCats.extend(cats['categories']['items'])
    offset += limit

In [43]:
len(allCats)


32

In [44]:
for cat in allCats:
    print(cat['name'])



Top Lists
Mood
Pop
Trending
Party
Focus
Rock
Indie
Electronic/Dance
Chill
Dinner
Sleep
Hip Hop
Workout
RnB
Country
Folk & Americana
Metal
Soul
Travel
Decades
Jazz
Blues
Reggae
Latino
Punk
Romance
Funk
Classical
Comedy
Kids
Gaming


Interesting that there are only 32. 

I'm not sure if there are more / different categories for other people - or what.

## Playlists

Next we get playlists associated with each category.

In [45]:
def get_playlists(category):
    all_playlists = []
    # here we could expand the range
    # or better, be smarter and see if 
    # the query has increased our playlist count
    # and only stop when there are no more.
    # 
    # for now, 100 playlists seems plenty. 
    for i in range(2):
        limit = 50
        offset = 0
        playlists = sp.category_playlists(category_id = category['id'], limit = limit, offset = offset)    
        all_playlists.extend(playlists['playlists']['items'])
    return all_playlists





In [46]:
playlists = get_playlists(allCats[0])

In [47]:
# we can see that not every category is 
# going to have 100 playlists
len(playlists)

26

In [161]:
playlists[0]

{'collaborative': False,
 'external_urls': {'spotify': 'http://open.spotify.com/user/spotify/playlist/5FJXhjdILmRA2z5bvz4nzf'},
 'href': 'https://api.spotify.com/v1/users/spotify/playlists/5FJXhjdILmRA2z5bvz4nzf',
 'id': '5FJXhjdILmRA2z5bvz4nzf',
 'images': [{'height': None,
   'url': 'https://u.scdn.co/images/pl/default/4ac24e496cf8aa49b7354b5795713e3eb167ed76',
   'width': None}],
 'name': "Today's Top Hits",
 'owner': {'external_urls': {'spotify': 'http://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'public': None,
 'snapshot_id': '+lOZPvXyYolqPrA8SG4t309l3ElBSgP7KSPqj9ga4WWJHLaJvwpCcCnNbR0ujKca',
 'tracks': {'href': 'https://api.spotify.com/v1/users/spotify/playlists/5FJXhjdILmRA2z5bvz4nzf/tracks',
  'total': 50},
 'type': 'playlist',
 'uri': 'spotify:user:spotify:playlist:5FJXhjdILmRA2z5bvz4nzf'}

Get playlists for all categories. 

In [49]:
for cat in allCats:
    cat['playlists'] = get_playlists(cat)

In [51]:
with open('data/categories.json', 'w') as outfile:
    json.dump(allCats, outfile)

In [54]:
for cat in allCats:
    print(cat['name'] + ' ' + str(len(cat['playlists'])))

Top Lists 26
Mood 58
Pop 28
Trending 34
Party 36
Focus 30
Rock 54
Indie 50
Electronic/Dance 100
Chill 70
Dinner 34
Sleep 18
Hip Hop 68
Workout 50
RnB 48
Country 38
Folk & Americana 38
Metal 82
Soul 30
Travel 34
Decades 36
Jazz 52
Blues 28
Reggae 26
Latino 100
Punk 16
Romance 46
Funk 34
Classical 100
Comedy 12
Kids 22
Gaming 72


Exploring some of the features of a playlist

In [56]:
playlist = allCats[0]['playlists'][0]

In [64]:
playlist

{'collaborative': False,
 'external_urls': {'spotify': 'http://open.spotify.com/user/spotify/playlist/5FJXhjdILmRA2z5bvz4nzf'},
 'href': 'https://api.spotify.com/v1/users/spotify/playlists/5FJXhjdILmRA2z5bvz4nzf',
 'id': '5FJXhjdILmRA2z5bvz4nzf',
 'images': [{'height': None,
   'url': 'https://u.scdn.co/images/pl/default/4ac24e496cf8aa49b7354b5795713e3eb167ed76',
   'width': None}],
 'name': "Today's Top Hits",
 'owner': {'external_urls': {'spotify': 'http://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'public': None,
 'snapshot_id': '+lOZPvXyYolqPrA8SG4t309l3ElBSgP7KSPqj9ga4WWJHLaJvwpCcCnNbR0ujKca',
 'tracks': {'href': 'https://api.spotify.com/v1/users/spotify/playlists/5FJXhjdILmRA2z5bvz4nzf/tracks',
  'total': 50},
 'type': 'playlist',
 'uri': 'spotify:user:spotify:playlist:5FJXhjdILmRA2z5bvz4nzf'}

In [58]:
playlist['tracks']

{'href': 'https://api.spotify.com/v1/users/spotify/playlists/5FJXhjdILmRA2z5bvz4nzf/tracks',
 'total': 50}

In [162]:
# here i figure out how to get a playlists' tracks
tracks = sp.user_playlist_tracks(playlist['owner']['id'], playlist['id'])


In [75]:
tracks['items'][0]['track']['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1RyvyyTE3xzB2ZywiAwp0i'},
  'href': 'https://api.spotify.com/v1/artists/1RyvyyTE3xzB2ZywiAwp0i',
  'id': '1RyvyyTE3xzB2ZywiAwp0i',
  'name': 'Future',
  'type': 'artist',
  'uri': 'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
  'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
  'id': '5pKCCKE2ajJHZ9KAiaK11H',
  'name': 'Rihanna',
  'type': 'artist',
  'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}]

## Tracks and Artists

Use what we've learned about playlists to pull out all the artists associated with the playlists

In [114]:
def get_artists(playlist):
    tracks = sp.user_playlist_tracks(playlist['owner']['id'], playlist['id'])
    all_artists = []
    if 'items' not in tracks:
        print('NO TRACKS')
        return []
    for track in tracks['items']:
        if track and 'track' in track and track['track'] and 'artists' in track['track']:
            artists = track['track']['artists']
            
            for artist in artists:
                all_artists.append(artist)
    return all_artists
            
        
    

In [115]:
artists = get_artists(playlist)

In [163]:

artists[0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2vDV0T8sxx2ENnKXds75e5'},
 'href': 'https://api.spotify.com/v1/artists/2vDV0T8sxx2ENnKXds75e5',
 'id': '2vDV0T8sxx2ENnKXds75e5',
 'name': "Booker T. & the M.G.'s",
 'type': 'artist',
 'uri': 'spotify:artist:2vDV0T8sxx2ENnKXds75e5'}

Get artists for all playlists

In [117]:
for cat in allCats:
    all_artists = []
    for playlist in cat['playlists']:
        artists = get_artists(playlist)
        all_artists.extend(artists)
    cat['artists'] = all_artists

In [118]:
with open('data/artists.json', 'w') as outfile:
    json.dump(allCats, outfile)

In [119]:
for cat in allCats:
    print(cat['name'] + ' ' + str(len(cat['artists'])))

Top Lists 1862
Mood 5182
Pop 2730
Trending 2624
Party 3858
Focus 2898
Rock 4084
Indie 3650
Electronic/Dance 12974
Chill 6154
Dinner 2798
Sleep 1316
Hip Hop 5976
Workout 5514
RnB 3202
Country 2362
Folk & Americana 3482
Metal 4656
Soul 2058
Travel 2352
Decades 2626
Jazz 3608
Blues 1286
Reggae 1698
Latino 7778
Punk 874
Romance 4076
Funk 2034
Classical 17480
Comedy 822
Kids 1656
Gaming 5742


In [120]:
allCats[0]['artists'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1RyvyyTE3xzB2ZywiAwp0i'},
 'href': 'https://api.spotify.com/v1/artists/1RyvyyTE3xzB2ZywiAwp0i',
 'id': '1RyvyyTE3xzB2ZywiAwp0i',
 'name': 'Future',
 'type': 'artist',
 'uri': 'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i'}

In [121]:
# pull out sublist of unique artist objects 
# from a list of artists
def unique_artists(artists):
    artist_ids = []
    uniq_artists = []
    for artist in artists:
        if artist['id'] not in artist_ids:
            uniq_artists.append(artist)
            artist_ids.append(artist['id'])
    return uniq_artists

Compare unique artist count with raw artist count.


we will use `uniq_artists` for the rest of the script

In [122]:
for cat in allCats:
    cat['uniq_artists'] = unique_artists(cat['artists'])

In [166]:
for cat in allCats:
    print(cat['name'] + ' -- all artists: ' + str(len(cat['artists'])) + ' unique artists: ' + str(len(cat['uniq_artists'])))

Top Lists -- all artists: 1862 unique artists: 544
Mood -- all artists: 5182 unique artists: 1907
Pop -- all artists: 2730 unique artists: 673
Trending -- all artists: 2624 unique artists: 1000
Party -- all artists: 3858 unique artists: 1113
Focus -- all artists: 2898 unique artists: 852
Rock -- all artists: 4084 unique artists: 1098
Indie -- all artists: 3650 unique artists: 1172
Electronic/Dance -- all artists: 12974 unique artists: 3330
Chill -- all artists: 6154 unique artists: 2026
Dinner -- all artists: 2798 unique artists: 947
Sleep -- all artists: 1316 unique artists: 297
Hip Hop -- all artists: 5976 unique artists: 1213
Workout -- all artists: 5514 unique artists: 1500
RnB -- all artists: 3202 unique artists: 690
Country -- all artists: 2362 unique artists: 443
Folk & Americana -- all artists: 3482 unique artists: 1044
Metal -- all artists: 4656 unique artists: 1209
Soul -- all artists: 2058 unique artists: 711
Travel -- all artists: 2352 unique artists: 873
Decades -- all art

## Artist Details

Finally, we can pull down the artist details for each category.

Since the `artists` API call can handle a max of 50 artist ids, we chunk up IDs.

In [126]:
# http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [154]:
def chunk_artists(artists):
    c = list(chunks(artists, 50))
    return c

def get_artist_details(artists):
    artist_chunks = chunk_artists(artists)
    all_details = []
    print(len(artists))
    for artist_chunk in artist_chunks:
        #print(len(artist_chunk))
        artist_ids = [a['id'] for a in artist_chunk if a['id'] is not None]
        

        deets = sp.artists(artist_ids)
        all_details.extend(deets['artists'])
        
    return all_details
        

In [170]:
deets = get_artist_details(allCats[0]['uniq_artists'])



544


In [171]:
deets[0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1RyvyyTE3xzB2ZywiAwp0i'},
 'followers': {'href': None, 'total': 2393953},
 'genres': ['dwn trap', 'pop rap', 'rap', 'southern hip hop', 'trap music'],
 'href': 'https://api.spotify.com/v1/artists/1RyvyyTE3xzB2ZywiAwp0i',
 'id': '1RyvyyTE3xzB2ZywiAwp0i',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/73fe3d7939e5bf6f37bebc523b8d5b9df2989e6f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/b9116e3539344bfe22c360facb6b42741c5ab9dc',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/6aab6547fff2b64e9a36d99544d6e6c2e921acf8',
   'width': 160}],
 'name': 'Future',
 'popularity': 93,
 'type': 'artist',
 'uri': 'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i'}

In [158]:
for cat in allCats:
    if 'artist_details' not in cat:
        cat['artist_details'] = get_artist_details(cat['uniq_artists'])

1058
268
1363
608
3784
108
473
1945


In [167]:
for cat in allCats:
    print(cat['name'] + ' -- artist details: ' + str(len(cat['artist_details'])) + ' uniq artists: ' + str(len(cat['uniq_artists'])))

Top Lists -- artist details: 544 uniq artists: 544
Mood -- artist details: 1907 uniq artists: 1907
Pop -- artist details: 672 uniq artists: 673
Trending -- artist details: 1000 uniq artists: 1000
Party -- artist details: 1112 uniq artists: 1113
Focus -- artist details: 852 uniq artists: 852
Rock -- artist details: 1098 uniq artists: 1098
Indie -- artist details: 1172 uniq artists: 1172
Electronic/Dance -- artist details: 3330 uniq artists: 3330
Chill -- artist details: 2026 uniq artists: 2026
Dinner -- artist details: 947 uniq artists: 947
Sleep -- artist details: 297 uniq artists: 297
Hip Hop -- artist details: 1213 uniq artists: 1213
Workout -- artist details: 1500 uniq artists: 1500
RnB -- artist details: 690 uniq artists: 690
Country -- artist details: 443 uniq artists: 443
Folk & Americana -- artist details: 1044 uniq artists: 1044
Metal -- artist details: 1209 uniq artists: 1209
Soul -- artist details: 711 uniq artists: 711
Travel -- artist details: 872 uniq artists: 873
Decades 

In [160]:
with open('data/artist_details.json', 'w') as outfile:
    json.dump(allCats, outfile)

## Images

In [173]:
deet = allCats[0]['artist_details'][0]

In [176]:
len(deet['images'])

3

In [184]:
print(len(allCats[0]['artist_details']))
more_images = [d for d in allCats[0]['artist_details'] if len(d['images']) < 1]

print(len(more_images))

544
35


In [180]:
more_images[0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
 'followers': {'href': None, 'total': 9628464},
 'genres': ['dance pop',
  'pop',
  'pop christmas',
  'r&b',
  'soul christmas',
  'urban contemporary'],
 'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
 'id': '5pKCCKE2ajJHZ9KAiaK11H',
 'images': [{'height': 667,
   'url': 'https://i.scdn.co/image/7a14a83633fc6c00c3977d134397b7235f888cdf',
   'width': 1000},
  {'height': 427,
   'url': 'https://i.scdn.co/image/ca67ea77cd79d6909c966870ae6f53297baba0bf',
   'width': 640},
  {'height': 133,
   'url': 'https://i.scdn.co/image/5376e8a81d2849ed25cecc9b55789f91c7cb4269',
   'width': 200},
  {'height': 43,
   'url': 'https://i.scdn.co/image/2bfed337f30cb4aedc26c39fc10fcce66a925b5b',
   'width': 64}],
 'name': 'Rihanna',
 'popularity': 90,
 'type': 'artist',
 'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'}

In [187]:
urllib.request.urlretrieve("http://www.gunnerkrigg.com//comics/00000001.jpg", "data/imgs/00000001.jpg")

('data/imgs/00000001.jpg', <http.client.HTTPMessage at 0x113d8f668>)

In [202]:

def get_url(artist):
    url = None
    if len(artist['images']) > 0:
        url = artist['images'][0]['url']
    return url
        
    

def download_images(category, artists):
    
    cat_dir = "data/imgs/" + category
    if not os.path.isdir(cat_dir):
        os.mkdir(cat_dir)
    for artist in artists:
        if len(artist['images']) > 0:
            url = get_url(artist)
            artist_id = artist['id']
            path = cat_dir + "/" + artist_id + ".jpg"
            urllib.request.urlretrieve(url, path)
            
            
    
                                   
            

In [195]:
for cat in allCats:
    download_images(cat['id'], cat['artist_details'])

## Face Features

In [197]:
interesting_ids = ['rock', 'indie_alt', 'pop', 'hiphop', 'rnb', 'country', 'folk_americana', 'metal', 'soul', 'punk']

inCats = [cat for cat in allCats if cat['id'] in interesting_ids]

In [199]:
total_artists = 0
for cat in inCats:
    print(cat['name'] + ' -- artist details: ' + str(len(cat['artist_details'])) + ' uniq artists: ' + str(len(cat['uniq_artists'])))
    total_artists += len(cat['artist_details'])
print("total artists: " + str(total_artists))

Pop -- artist details: 672 uniq artists: 673
Rock -- artist details: 1098 uniq artists: 1098
Indie -- artist details: 1172 uniq artists: 1172
Hip Hop -- artist details: 1213 uniq artists: 1213
RnB -- artist details: 690 uniq artists: 690
Country -- artist details: 443 uniq artists: 443
Folk & Americana -- artist details: 1044 uniq artists: 1044
Metal -- artist details: 1209 uniq artists: 1209
Soul -- artist details: 711 uniq artists: 711
Punk -- artist details: 268 uniq artists: 268
total artists: 8520


In [216]:
with open('data/interesting_artist_details.json', 'w') as outfile:
    json.dump(inCats, outfile)

In [214]:
########### Python 3.2 #############


M_KEY = '7223b90581904d7b834d2319e5087a2d'


import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': M_KEY,
}

def get_faces(artist):
    url = get_url(artist)
    print(url)
    if not url:
        return {}

    params = urllib.parse.urlencode({
        # Request parameters
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'true',
        'returnFaceAttributes': 'age,gender,headPose,glasses,smile,facialHair',
    })
    
    data = {}
    #body = "{\"url\": "+ url + "}"
    body = {"url": url}

    try:
        conn = http.client.HTTPSConnection('westus.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/detect?%s" % params, json.dumps(body), headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return data

####################################

In [206]:
artist = inCats[0]['artist_details'][0]
artist

{'external_urls': {'spotify': 'https://open.spotify.com/artist/1RyvyyTE3xzB2ZywiAwp0i'},
 'followers': {'href': None, 'total': 2393474},
 'genres': ['dwn trap', 'pop rap', 'rap', 'southern hip hop', 'trap music'],
 'href': 'https://api.spotify.com/v1/artists/1RyvyyTE3xzB2ZywiAwp0i',
 'id': '1RyvyyTE3xzB2ZywiAwp0i',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/73fe3d7939e5bf6f37bebc523b8d5b9df2989e6f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/b9116e3539344bfe22c360facb6b42741c5ab9dc',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/6aab6547fff2b64e9a36d99544d6e6c2e921acf8',
   'width': 160}],
 'name': 'Future',
 'popularity': 93,
 'type': 'artist',
 'uri': 'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i'}

In [215]:
faces = get_faces(artist)

https://i.scdn.co/image/73fe3d7939e5bf6f37bebc523b8d5b9df2989e6f
b'[{"faceId":"397abede-0a70-4954-8703-0a712ad04878","faceRectangle":{"top":294,"left":189,"width":157,"height":157},"faceLandmarks":{"pupilLeft":{"x":236.0,"y":334.7},"pupilRight":{"x":302.8,"y":337.2},"noseTip":{"x":268.9,"y":376.0},"mouthLeft":{"x":235.9,"y":410.7},"mouthRight":{"x":293.5,"y":413.9},"eyebrowLeftOuter":{"x":209.6,"y":321.4},"eyebrowLeftInner":{"x":254.8,"y":318.7},"eyeLeftOuter":{"x":225.0,"y":336.1},"eyeLeftTop":{"x":236.9,"y":330.4},"eyeLeftBottom":{"x":236.8,"y":340.6},"eyeLeftInner":{"x":247.5,"y":336.1},"eyebrowRightInner":{"x":283.3,"y":321.8},"eyebrowRightOuter":{"x":328.6,"y":327.7},"eyeRightInner":{"x":293.5,"y":338.1},"eyeRightTop":{"x":304.6,"y":331.9},"eyeRightBottom":{"x":304.4,"y":342.4},"eyeRightOuter":{"x":314.5,"y":338.2},"noseRootLeft":{"x":258.1,"y":338.0},"noseRootRight":{"x":278.6,"y":339.1},"noseLeftAlarTop":{"x":255.8,"y":361.4},"noseRightAlarTop":{"x":282.9,"y":365.1},"noseLeftAla

In [ ]:
## Automated the rest of the analysis in ./get_faces.py